In [1]:
!nvidia-smi

Sat Nov 23 12:11:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.51.01              Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              1W /   60W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
SCRIPT_DIR = os.path.abspath("")

In [2]:
from dataset_util import merge

train_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/train'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/train'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/train'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/train')]
taco_index = [0,1]

merge.merge_yolo(train_sets, os.path.join(SCRIPT_DIR, 'merged_data/train'), taco_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')
yaml_content = """train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Person', 'Trash']
"""

with open(f"{output_dir}/data.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/train completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/train completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/train completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/train completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/train


In [4]:
validation_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/valid'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/valid'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/valid'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/valid')]
taco_index = [0,1]

merge.merge_yolo(validation_sets, os.path.join(SCRIPT_DIR, 'merged_data/valid'), taco_index)


Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/valid completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/valid completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/valid completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/valid completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid


In [6]:
test_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/test'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/test'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/test'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/test')]
taco_index = [0,1]

merge.merge_yolo(test_sets, os.path.join(SCRIPT_DIR, 'merged_data/test'), taco_index)


Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/test completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/test completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/test completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/test completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/test


In [3]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt") 
merged_data = os.path.join(SCRIPT_DIR, 'merged_data/data.yaml')
output_dir = os.path.join(SCRIPT_DIR, 'model')
results = model.train(name='Trash Detector', data=merged_data, epochs=100, 
                      imgsz=416, batch=32, device=0, project=output_dir, 
                      resume=False, plots=True, exist_ok=True,
                      save_period=10)

New https://pypi.org/project/ultralytics/8.3.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/mnt/c/Adrianov/Projects/trash-detector/merged_data/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=416, save=True, save_period=10, cache=False, device=0, workers=8, project=/mnt/c/Adrianov/Projects/trash-detector/model, name=Trash Detector, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,

train: Scanning /mnt/c/Adrianov/Projects/trash-detector/merged_data/train/labels... 14177 images, 51 backgrounds, 0 corrupt: 100%|██████████| 14


train: New cache created: /mnt/c/Adrianov/Projects/trash-detector/merged_data/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 14903, len(boxes) = 48825. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid/labels... 2902 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2902/2


val: New cache created: /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 4082, len(boxes) = 10159. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to /mnt/c/Adrianov/Projects/trash-detector/model/Trash Detector/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to /mnt/c/Adrianov/Projects/trash-detector/model/Trash Detector
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2902      10159      0.723      0.487       0.56      0.355

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     41/100      3.82G      1.247      1.097      1.168          2        416: 100%|██████████| 444/444 [02:20<00:00,  3.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:32<00:00,  1.43it/s]


                   all       2902      10159      0.728      0.482      0.562      0.356

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     42/100       2.9G      1.252      1.094      1.171        179        416:  21%|██        | 94/444 [00:30<01:53,  3.10it/s]


KeyboardInterrupt: 